In [10]:
import pandas as pd
import numpy as np
import nrrd
import SimpleITK as sitk
import os
import six
from radiomics import featureextractor, getTestCase

In [14]:
xlsx_filepath = 'D:/Desktop/data.xls'
data = pd.read_excel(xlsx_filepath)

In [22]:
data.head()

,姓名,住院号,是否pCR,TRG,病理,基线T分期,基线N分期,基线系膜内淋巴结,基线系膜外淋巴结,基线CRM,基线肿瘤体积,手术T分期,手术N分期,病理淋巴结,术后系膜外淋巴结,术后系膜内淋巴结,术后CRM,病理肿瘤体积,肿瘤体积缩小百分比
0,孙跃珍,16483567,0,3,直肠癌病理评估表大体检查肿瘤体积：3cm×2cm×1.5cm；肿瘤大体分型：溃疡型。 显微镜...,4b,1,1,1,1,35.696323,3,0,(-),0,0,0,9.00,0.747873
1,邵裴华,465981,0,3,直肠肿瘤病理评估表大体检查肿瘤体积：6.5cm×3.5cm×1.2cm；肿瘤大体分型：隆起型...,4a,2,1,1,1,85.826957,3,2,“淋巴结”（5/9）枚，查见癌转移。,0,1,0,27.30,0.681918
2,蒋兴发,16657839,0,2,直肠肿瘤病理评估表大体检查肿瘤体积：3cm×2cm×1.5cm；肿瘤大体分型：溃疡型。肿瘤部...,4a,2,1,1,0,27.335066,3,1,肠周脂肪内扪及淋巴结（1/3）枚，查见癌转移。,0,1,0,9.00,0.670753
3,程香群,16710554,0,3,直肠肿瘤病理评估表大体检查肿瘤体积：4cm×3.5cm×2cm。肿瘤大体分型：溃疡型。肿瘤部...,3,2,1,1,1,87.708712,3,0,(-),0,0,0,28.00,0.680761
4,杨华琼,15365389,0,3,大体检查肿瘤体积：3.5cm×3cm×1.5cm；肿瘤大体分型：溃疡型。肿瘤部位：累及肠管全...,4b,2,1,1,1,23.845006,3,1,肠周脂肪内查见癌结节,0,1,1,15.75,0.339484


In [11]:
maskpath = 'D:/Desktop/mask.nrrd'
data, header = nrrd.read(maskpath)

In [17]:
#np.where(condition, x, y)
#满足条件(condition)，输出x，不满足输出y。
cancer = np.where(data[:, :, :] == 1, 1, 0)
print(cancer.shape)
nrrd.write('D:\Desktop\ms\mask.nrrd', cancer, header)

(768, 696, 48)


In [20]:
# # get image
# series_IDs = sitk.ImageSeriesReader.GetGDCMSeriesIDs(image_path)
# nb_series = len(series_IDs)
# print(nb_series)
# series_file_names = sitk.ImageSeriesReader.GetGDCMSeriesFileNames(image_path,series_IDs[0])
# series_reader = sitk.ImageSeriesReader()
# series_reader.SetFileNames(series_file_names)
# image3D = series_reader.Execute()
image = sitk.ReadImage('D:\Desktop\ms\syz.nrrd')
img_array = sitk.GetArrayFromImage(image)
print(img_array.shape)
# get mask
mask = sitk.ReadImage('D:\Desktop\ms\mask.nrrd')
img = sitk.GetArrayFromImage(mask)
print(img.shape)

(48, 696, 768)
(48, 696, 768)


In [21]:
# dataDir = 'D:\Desktop\ms'
# image = '\syz.nrrd'
# mask = '\mask.nrrd'
Params = 'D:\Desktop\ms\Params.yaml'

In [26]:
extractor = featureextractor.RadiomicsFeatureExtractor(geometryTolerance=0.001)
result = extractor.execute(image, mask)
features_dict = dict()
df = pd.DataFrame()
for key, val in six.iteritems(result):
    if 'diagnostics_Versions' in  key or 'diagnostics_Configuration' in key:#这些都是一些共有的特征，可以去掉
        continue
    features_dict[key] = val
df = df.append(pd.DataFrame.from_dict(features_dict.values()).T,ignore_index=True)
#print(index)
df.columns = features_dict.keys()
df.to_csv('log.csv',index=0)

GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


In [31]:
data = pd.read_csv('log.csv')
data.head()

,diagnostics_Image-original_Hash,diagnostics_Image-original_Dimensionality,diagnostics_Image-original_Spacing,diagnostics_Image-original_Size,diagnostics_Image-original_Mean,diagnostics_Image-original_Minimum,diagnostics_Image-original_Maximum,diagnostics_Mask-original_Hash,diagnostics_Mask-original_Spacing,diagnostics_Mask-original_Size,...,original_glszm_SmallAreaHighGrayLevelEmphasis,original_glszm_SmallAreaLowGrayLevelEmphasis,original_glszm_ZoneEntropy,original_glszm_ZonePercentage,original_glszm_ZoneVariance,original_ngtdm_Busyness,original_ngtdm_Coarseness,original_ngtdm_Complexity,original_ngtdm_Contrast,original_ngtdm_Strength
0,a295bcaa408544f315981818ac348b4235688bbe,3D,"(0.33854165673256, 0.3385416567325601, 3.89999...","(768, 696, 48)",275.04743,0.0,2282.0,93401692dab92359a99b8177cfb6242dfeb8d2f2,"(0.3385416567325592, 0.33854165673255926, 3.89...","(768, 696, 48)",...,260.411824,0.003335,7.158345,0.137489,21368.078031,9.797537,0.000103,2802.846562,0.031979,0.301203
